In [ ]:
def is_empty(board):
    for i in board:
        for j in i:
            if j != " ":
                return False
    return True

def is_full(board):
    for i in board:
          for j in i:
              if j == " ":
                  return False
    return True

def is_bounded(board, y_end, x_end, length, d_y, d_x):
    board_size = len(board)
    y_start = y_end - (length - 1) * d_y
    x_start = x_end - (length - 1) * d_x

    start_in_bounds = 0 <= y_start < board_size and 0 <= x_start < board_size
    end_in_bounds = 0 <= y_end < board_size and 0 <= x_end < board_size

    if not start_in_bounds and not end_in_bounds:
        return "CLOSED"

    y_before = y_start - d_y
    x_before = x_start - d_x
    y_after = y_end + d_y
    x_after = x_end + d_x

    before_in_bounds = 0 <= y_before < board_size and 0 <= x_before < board_size
    after_in_bounds = 0 <= y_after < board_size and 0 <= x_after < board_size

    open_ends = 0
    if before_in_bounds and board[y_before][x_before] == " " and after_in_bounds and board[y_after][x_after] == " ":
        return "OPEN"
    elif (before_in_bounds and board[y_before][x_before] == " ") or (after_in_bounds and board[y_after][x_after] == " "):
        return "SEMIOPEN"
    else:
        return "CLOSED"


def detect_row(board, col, y_start, x_start, length, d_y, d_x):
    open_seq_count, semi_open_seq_count = 0, 0
    y = y_start
    x = x_start
    while 0 <= y < len(board) and 0 <= x < len(board):
        if is_sequence_complete(board, y, x, d_y, d_x, length, col):
            y_end = y + (length - 1) * d_y
            x_end = x + (length - 1) * d_x
            bounded_status = is_bounded(board, y_end, x_end, length, d_y, d_x)
            if bounded_status == "OPEN":
                open_seq_count += 1
            elif bounded_status == "SEMIOPEN":
                semi_open_seq_count += 1
            y += length * d_y
            x += length * d_x
        else:
            y += d_y
            x += d_x

    return open_seq_count, semi_open_seq_count

def detect_rows(board, col, length):
    open_seq_count, semi_open_seq_count = 0, 0
    for y in range(len(board)):
        for x in range(len(board)):
            if board[y][x] == col:
                for d_y, d_x in [(0, 1), (1, 0), (1, 1), (1, -1)]:
                     open, semi_open = detect_row(board, col, y, x, length, d_y, d_x)
                     open_seq_count += open
                     semi_open_seq_count += semi_open

    return open_seq_count, semi_open_seq_count

def search_max(board):
    max_x = 0
    max_y = 0
    max_score = -1e100

    for y in range(len(board)):
        for x in range(len(board)):
            if board[y][x] == " ":
                test = board[:][:]
                test[y][x] = "b"
                score_i = score(test)
                if score_i > max_score:
                    max_score = score_i
                    max_y = y
                    max_x = x

    return max_y, max_x


def score(board):
    MAX_SCORE = 100000

    open_b = {}
    semi_open_b = {}
    open_w = {}
    semi_open_w = {}

    for i in range(2, 6):
        open_b[i], semi_open_b[i] = detect_rows(board, "b", i)
        open_w[i], semi_open_w[i] = detect_rows(board, "w", i)


    if open_b[5] >= 1 or semi_open_b[5] >= 1:
        return MAX_SCORE

    elif open_w[5] >= 1 or semi_open_w[5] >= 1:
        return -MAX_SCORE

    return (-10000 * (open_w[4] + semi_open_w[4])+
            500  * open_b[4]                     +
            50   * semi_open_b[4]                +
            -100  * open_w[3]                    +
            -30   * semi_open_w[3]               +
            50   * open_b[3]                     +
            10   * semi_open_b[3]                +
            open_b[2] + semi_open_b[2] - open_w[2] - semi_open_w[2])


def is_win(board):
    if is_full(board):
      return "Draw"
    for i in board:
      for j in i:
        if is_sequence_complete(board, i, j, 1, 0, 5, "w") or is_sequence_complete(board, i, j, 0, 1, 5, "w") or is_sequence_complete(board, i, j, 1, 1, 5, "w") or is_sequence_complete(board, i, j, -1, 1, 5, "w"):
          return "White won"
        elif is_sequence_complete(board, i, j, 1, 0, 5, "b") or is_sequence_complete(board, i, j, 0, 1, 5, "b") or is_sequence_complete(board, i, j, 1, 1, 5, "b") or is_sequence_complete(board, i, j, -1, 1, 5, "b"):
          return "Black won"
    return "Continue playing"

def print_board(board):

    s = "*"
    for i in range(len(board[0])-1):
        s += str(i%10) + "|"
    s += str((len(board[0])-1)%10)
    s += "*\n"

    for i in range(len(board)):
        s += str(i%10)
        for j in range(len(board[0])-1):
            s += str(board[i][j]) + "|"
        s += str(board[i][len(board[0])-1])

        s += "*\n"
    s += (len(board[0])*2 + 1)*"*"

    print(s)


def make_empty_board(sz):
    board = []
    for i in range(sz):
        board.append([" "]*sz)
    return board



def analysis(board):
    for c, full_name in [["b", "Black"], ["w", "White"]]:
        print("%s stones" % (full_name))
        for i in range(2, 6):
            open, semi_open = detect_rows(board, c, i);
            print("Open rows of length %d: %d" % (i, open))
            print("Semi-open rows of length %d: %d" % (i, semi_open))






def play_gomoku(board_size):
    board = make_empty_board(board_size)
    board_height = len(board)
    board_width = len(board[0])

    while True:
        print_board(board)
        if is_empty(board):
            move_y = board_height // 2
            move_x = board_width // 2
        else:
            move_y, move_x = search_max(board)

        print("Computer move: (%d, %d)" % (move_y, move_x))
        board[move_y][move_x] = "b"
        print_board(board)
        analysis(board)

        game_res = is_win(board)
        if game_res in ["White won", "Black won", "Draw"]:
            return game_res





        print("Your move:")
        move_y = int(input("y coord: "))
        move_x = int(input("x coord: "))
        board[move_y][move_x] = "w"
        print_board(board)
        analysis(board)

        game_res = is_win(board)
        if game_res in ["White won", "Black won", "Draw"]:
            return game_res



def put_seq_on_board(board, y, x, d_y, d_x, length, col):
    for i in range(length):
        board[y][x] = col
        y += d_y
        x += d_x

def is_sequence_complete(board, y, x, d_y, d_x, length, col):
    for i in range(length):
        if not (0 <= y < len(board) and 0 <= x < len(board)) or board[y][x] != col:
          return False
        y += d_y
        x += d_x
    return True


def test_is_empty():
    board  = make_empty_board(8)
    if is_empty(board):
        print("TEST CASE for is_empty PASSED")
    else:
        print("TEST CASE for is_empty FAILED")

def test_is_bounded():
    board = make_empty_board(8)
    x = 5; y = 0; d_x = 0; d_y = 1; length = 4
    put_seq_on_board(board, y, x, d_y, d_x, length, "w")
    print_board(board)

    y_end = 3
    x_end = 5

    if is_bounded(board, y_end, x_end, length, d_y, d_x) == 'SEMIOPEN':
        print("TEST CASE for is_bounded PASSED")
    else:
        print("TEST CASE for is_bounded FAILED")


def test_detect_row():
    board = make_empty_board(8)
    x = 5; y = 1; d_x = 0; d_y = 1; length = 3
    put_seq_on_board(board, y, x, d_y, d_x, length, "w")
    print_board(board)
    if detect_row(board, "w", 0,x,length,d_y,d_x) == (1,0):
        print("TEST CASE for detect_row PASSED")
    else:
        print("TEST CASE for detect_row FAILED")

def test_detect_rows():
    board = make_empty_board(8)
    x = 5; y = 1; d_x = 0; d_y = 1; length = 3; col = 'w'
    put_seq_on_board(board, y, x, d_y, d_x, length, "w")
    print_board(board)
    if detect_rows(board, col,length) == (1,0):
        print("TEST CASE for detect_rows PASSED")
    else:
        print("TEST CASE for detect_rows FAILED")

def test_search_max():
    board = make_empty_board(8)
    x = 5; y = 0; d_x = 0; d_y = 1; length = 4; col = 'w'
    put_seq_on_board(board, y, x, d_y, d_x, length, col)
    x = 6; y = 0; d_x = 0; d_y = 1; length = 4; col = 'b'
    put_seq_on_board(board, y, x, d_y, d_x, length, col)
    print_board(board)
    print(search_max(board))
    if search_max(board) == (4,6):
        print("TEST CASE for search_max PASSED")
    else:
        print("TEST CASE for search_max FAILED")

def easy_testset_for_main_functions():
    test_is_empty()
    test_is_bounded()
    test_detect_row()
    test_detect_rows()
    test_search_max()

def some_tests():
    board = make_empty_board(8)

    board[0][5] = "w"
    board[0][6] = "b"
    y = 5; x = 2; d_x = 0; d_y = 1; length = 3
    put_seq_on_board(board, y, x, d_y, d_x, length, "w")
    print_board(board)
    analysis(board)

    # Expected output:
    #       *0|1|2|3|4|5|6|7*
    #       0 | | | | |w|b| *
    #       1 | | | | | | | *
    #       2 | | | | | | | *
    #       3 | | | | | | | *
    #       4 | | | | | | | *
    #       5 | |w| | | | | *
    #       6 | |w| | | | | *
    #       7 | |w| | | | | *
    #       *****************
    #       Black stones:
    #       Open rows of length 2: 0
    #       Semi-open rows of length 2: 0
    #       Open rows of length 3: 0
    #       Semi-open rows of length 3: 0
    #       Open rows of length 4: 0
    #       Semi-open rows of length 4: 0
    #       Open rows of length 5: 0
    #       Semi-open rows of length 5: 0
    #       White stones:
    #       Open rows of length 2: 0
    #       Semi-open rows of length 2: 0
    #       Open rows of length 3: 0
    #       Semi-open rows of length 3: 1
    #       Open rows of length 4: 0
    #       Semi-open rows of length 4: 0
    #       Open rows of length 5: 0
    #       Semi-open rows of length 5: 0

    y = 3; x = 5; d_x = -1; d_y = 1; length = 2

    put_seq_on_board(board, y, x, d_y, d_x, length, "b")
    print_board(board)
    analysis(board)

    # Expected output:
    #        *0|1|2|3|4|5|6|7*
    #        0 | | | | |w|b| *
    #        1 | | | | | | | *
    #        2 | | | | | | | *
    #        3 | | | | |b| | *
    #        4 | | | |b| | | *
    #        5 | |w| | | | | *
    #        6 | |w| | | | | *
    #        7 | |w| | | | | *
    #        *****************
    #
    #         Black stones:
    #         Open rows of length 2: 1
    #         Semi-open rows of length 2: 0
    #         Open rows of length 3: 0
    #         Semi-open rows of length 3: 0
    #         Open rows of length 4: 0
    #         Semi-open rows of length 4: 0
    #         Open rows of length 5: 0
    #         Semi-open rows of length 5: 0
    #         White stones:
    #         Open rows of length 2: 0
    #         Semi-open rows of length 2: 0
    #         Open rows of length 3: 0
    #         Semi-open rows of length 3: 1
    #         Open rows of length 4: 0
    #         Semi-open rows of length 4: 0
    #         Open rows of length 5: 0
    #         Semi-open rows of length 5: 0
    #

    y = 5; x = 3; d_x = -1; d_y = 1; length = 1
    put_seq_on_board(board, y, x, d_y, d_x, length, "b");
    print_board(board);
    analysis(board);

    #        Expected output:
    #           *0|1|2|3|4|5|6|7*
    #           0 | | | | |w|b| *
    #           1 | | | | | | | *
    #           2 | | | | | | | *
    #           3 | | | | |b| | *
    #           4 | | | |b| | | *
    #           5 | |w|b| | | | *
    #           6 | |w| | | | | *
    #           7 | |w| | | | | *
    #           *****************
    #
    #
    #        Black stones:
    #        Open rows of length 2: 0
    #        Semi-open rows of length 2: 0
    #        Open rows of length 3: 0
    #        Semi-open rows of length 3: 1
    #        Open rows of length 4: 0
    #        Semi-open rows of length 4: 0
    #        Open rows of length 5: 0
    #        Semi-open rows of length 5: 0
    #        White stones:
    #        Open rows of length 2: 0
    #        Semi-open rows of length 2: 0
    #        Open rows of length 3: 0
    #        Semi-open rows of length 3: 1
    #        Open rows of length 4: 0
    #        Semi-open rows of length 4: 0
    #        Open rows of length 5: 0
    #        Semi-open rows of length 5: 0




if __name__ == '__main__':
    easy_testset_for_main_functions()
    some_tests()

TEST CASE for is_empty PASSED
*0|1|2|3|4|5|6|7*
0 | | | | |w| | *
1 | | | | |w| | *
2 | | | | |w| | *
3 | | | | |w| | *
4 | | | | | | | *
5 | | | | | | | *
6 | | | | | | | *
7 | | | | | | | *
*****************
TEST CASE for is_bounded PASSED
*0|1|2|3|4|5|6|7*
0 | | | | | | | *
1 | | | | |w| | *
2 | | | | |w| | *
3 | | | | |w| | *
4 | | | | | | | *
5 | | | | | | | *
6 | | | | | | | *
7 | | | | | | | *
*****************
TEST CASE for detect_row PASSED
*0|1|2|3|4|5|6|7*
0 | | | | | | | *
1 | | | | |w| | *
2 | | | | |w| | *
3 | | | | |w| | *
4 | | | | | | | *
5 | | | | | | | *
6 | | | | | | | *
7 | | | | | | | *
*****************
TEST CASE for detect_rows PASSED
*0|1|2|3|4|5|6|7*
0 | | | | |w|b| *
1 | | | | |w|b| *
2 | | | | |w|b| *
3 | | | | |w|b| *
4 | | | | | | | *
5 | | | | | | | *
6 | | | | | | | *
7 | | | | | | | *
*****************
(4, 0)
TEST CASE for search_max FAILED
*0|1|2|3|4|5|6|7*
0 | | | | |w|b| *
1 | | | | | | | *
2 | | | | | | | *
3 | | | | | | | *
4 | | | | | | | *
5 | |w

In [ ]:
def is_empty(board):
    for i in board:
        for j in i:
            if j != " ":
                return False
    return True

def is_full(board):
    for i in board:
          for j in i:
              if j == " ":
                  return False
    return True

def is_bounded(board, y_end, x_end, length, d_y, d_x):
    board_size = len(board)
    y_start = y_end - (length - 1) * d_y
    x_start = x_end - (length - 1) * d_x

    start_in_bounds = 0 <= y_start < board_size and 0 <= x_start < board_size
    end_in_bounds = 0 <= y_end < board_size and 0 <= x_end < board_size

    if not start_in_bounds and not end_in_bounds:
        return "CLOSED"

    y_before = y_start - d_y
    x_before = x_start - d_x
    y_after = y_end + d_y
    x_after = x_end + d_x

    before_in_bounds = 0 <= y_before < board_size and 0 <= x_before < board_size
    after_in_bounds = 0 <= y_after < board_size and 0 <= x_after < board_size

    open_ends = 0
    if before_in_bounds and board[y_before][x_before] == " " and after_in_bounds and board[y_after][x_after] == " ":
        return "OPEN"
    elif (before_in_bounds and board[y_before][x_before] == " ") or (after_in_bounds and board[y_after][x_after] == " "):
        return "SEMIOPEN"
    else:
        return "CLOSED"


def detect_row(board, col, y_start, x_start, length, d_y, d_x):
    open_seq_count, semi_open_seq_count = 0, 0
    y = y_start
    x = x_start
    while 0 <= y < len(board) and 0 <= x < len(board):
        if is_sequence_complete(board, y, x, d_y, d_x, length, col):
            y_end = y + (length - 1) * d_y
            x_end = x + (length - 1) * d_x
            bounded_status = is_bounded(board, y_end, x_end, length, d_y, d_x)
            if bounded_status == "OPEN":
                open_seq_count += 1
            elif bounded_status == "SEMIOPEN":
                semi_open_seq_count += 1
            y += length * d_y
            x += length * d_x
        else:
            y += d_y
            x += d_x

    return open_seq_count, semi_open_seq_count

def detect_rows(board, col, length):
    open_seq_count, semi_open_seq_count = 0, 0
    for y in range(len(board)):
        for x in range(len(board)):
            if board[y][x] == col:
                for d_y, d_x in [(0, 1), (1, 0), (1, 1), (1, -1)]:
                     open, semi_open = detect_row(board, col, y, x, length, d_y, d_x)
                     open_seq_count += open
                     semi_open_seq_count += semi_open

    return open_seq_count, semi_open_seq_count

def search_max(board):
    max_x = 0
    max_y = 0
    max_score = -1e100

    for y in range(len(board)):
        for x in range(len(board)):
            if board[y][x] == " ":
                test = board[:][:]
                test[y][x] = "b"
                score_i = score(test)
                if score_i > max_score:
                    max_score = score_i
                    max_y = y
                    max_x = x

    return max_y, max_x


def score(board):
    MAX_SCORE = 100000

    open_b = {}
    semi_open_b = {}
    open_w = {}
    semi_open_w = {}

    for i in range(2, 6):
        open_b[i], semi_open_b[i] = detect_rows(board, "b", i)
        open_w[i], semi_open_w[i] = detect_rows(board, "w", i)


    if open_b[5] >= 1 or semi_open_b[5] >= 1:
        return MAX_SCORE

    elif open_w[5] >= 1 or semi_open_w[5] >= 1:
        return -MAX_SCORE

    return (-10000 * (open_w[4] + semi_open_w[4])+
            500  * open_b[4]                     +
            50   * semi_open_b[4]                +
            -100  * open_w[3]                    +
            -30   * semi_open_w[3]               +
            50   * open_b[3]                     +
            10   * semi_open_b[3]                +
            open_b[2] + semi_open_b[2] - open_w[2] - semi_open_w[2])


def is_win(board):
    if is_full(board):
      return "Draw"
    for i in board:
      for j in i:
        if is_sequence_complete(board, i, j, 1, 0, 5, "w") or is_sequence_complete(board, i, j, 0, 1, 5, "w") or is_sequence_complete(board, i, j, 1, 1, 5, "w") or is_sequence_complete(board, i, j, -1, 1, 5, "w"):
          return "White won"
        elif is_sequence_complete(board, i, j, 1, 0, 5, "b") or is_sequence_complete(board, i, j, 0, 1, 5, "b") or is_sequence_complete(board, i, j, 1, 1, 5, "b") or is_sequence_complete(board, i, j, -1, 1, 5, "b"):
          return "Black won"
    return "Continue playing"

def print_board(board):

    s = "*"
    for i in range(len(board[0])-1):
        s += str(i%10) + "|"
    s += str((len(board[0])-1)%10)
    s += "*\n"

    for i in range(len(board)):
        s += str(i%10)
        for j in range(len(board[0])-1):
            s += str(board[i][j]) + "|"
        s += str(board[i][len(board[0])-1])

        s += "*\n"
    s += (len(board[0])*2 + 1)*"*"

    print(s)


def make_empty_board(sz):
    board = []
    for i in range(sz):
        board.append([" "]*sz)
    return board



def analysis(board):
    for c, full_name in [["b", "Black"], ["w", "White"]]:
        print("%s stones" % (full_name))
        for i in range(2, 6):
            open, semi_open = detect_rows(board, c, i);
            print("Open rows of length %d: %d" % (i, open))
            print("Semi-open rows of length %d: %d" % (i, semi_open))






def play_gomoku(board_size):
    board = make_empty_board(board_size)
    board_height = len(board)
    board_width = len(board[0])

    while True:
        print_board(board)
        if is_empty(board):
            move_y = board_height // 2
            move_x = board_width // 2
        else:
            move_y, move_x = search_max(board)

        print("Computer move: (%d, %d)" % (move_y, move_x))
        board[move_y][move_x] = "b"
        print_board(board)
        analysis(board)

        game_res = is_win(board)
        if game_res in ["White won", "Black won", "Draw"]:
            return game_res





        print("Your move:")
        move_y = int(input("y coord: "))
        move_x = int(input("x coord: "))
        board[move_y][move_x] = "w"
        print_board(board)
        analysis(board)

        game_res = is_win(board)
        if game_res in ["White won", "Black won", "Draw"]:
            return game_res



def put_seq_on_board(board, y, x, d_y, d_x, length, col):
    for i in range(length):
        board[y][x] = col
        y += d_y
        x += d_x

def is_sequence_complete(board, y, x, d_y, d_x, length, col):
    for i in range(length):
        if not (0 <= y < len(board) and 0 <= x < len(board)) or board[y][x] != col:
          return False
        y += d_y
        x += d_x
    return True


def test_is_empty():
    board  = make_empty_board(8)
    if is_empty(board):
        print("TEST CASE for is_empty PASSED")
    else:
        print("TEST CASE for is_empty FAILED")

def test_is_bounded():
    board = make_empty_board(8)
    x = 5; y = 0; d_x = 0; d_y = 1; length = 4
    put_seq_on_board(board, y, x, d_y, d_x, length, "w")
    print_board(board)

    y_end = 3
    x_end = 5

    if is_bounded(board, y_end, x_end, length, d_y, d_x) == 'SEMIOPEN':
        print("TEST CASE for is_bounded PASSED")
    else:
        print("TEST CASE for is_bounded FAILED")


def test_detect_row():
    board = make_empty_board(8)
    x = 5; y = 1; d_x = 0; d_y = 1; length = 3
    put_seq_on_board(board, y, x, d_y, d_x, length, "w")
    print_board(board)
    if detect_row(board, "w", 0,x,length,d_y,d_x) == (1,0):
        print("TEST CASE for detect_row PASSED")
    else:
        print("TEST CASE for detect_row FAILED")

def test_detect_rows():
    board = make_empty_board(8)
    x = 5; y = 1; d_x = 0; d_y = 1; length = 3; col = 'w'
    put_seq_on_board(board, y, x, d_y, d_x, length, "w")
    print_board(board)
    if detect_rows(board, col,length) == (1,0):
        print("TEST CASE for detect_rows PASSED")
    else:
        print("TEST CASE for detect_rows FAILED")

def test_search_max():
    board = make_empty_board(8)
    x = 5; y = 0; d_x = 0; d_y = 1; length = 4; col = 'w'
    put_seq_on_board(board, y, x, d_y, d_x, length, col)
    x = 6; y = 0; d_x = 0; d_y = 1; length = 4; col = 'b'
    put_seq_on_board(board, y, x, d_y, d_x, length, col)
    print_board(board)
    print(search_max(board))
    if search_max(board) == (4,6):
        print("TEST CASE for search_max PASSED")
    else:
        print("TEST CASE for search_max FAILED")

def easy_testset_for_main_functions():
    test_is_empty()
    test_is_bounded()
    test_detect_row()
    test_detect_rows()
    test_search_max()

def some_tests():
    board = make_empty_board(8)

    board[0][5] = "w"
    board[0][6] = "b"
    y = 5; x = 2; d_x = 0; d_y = 1; length = 3
    put_seq_on_board(board, y, x, d_y, d_x, length, "w")
    print_board(board)
    analysis(board)

    # Expected output:
    #       *0|1|2|3|4|5|6|7*
    #       0 | | | | |w|b| *
    #       1 | | | | | | | *
    #       2 | | | | | | | *
    #       3 | | | | | | | *
    #       4 | | | | | | | *
    #       5 | |w| | | | | *
    #       6 | |w| | | | | *
    #       7 | |w| | | | | *
    #       *****************
    #       Black stones:
    #       Open rows of length 2: 0
    #       Semi-open rows of length 2: 0
    #       Open rows of length 3: 0
    #       Semi-open rows of length 3: 0
    #       Open rows of length 4: 0
    #       Semi-open rows of length 4: 0
    #       Open rows of length 5: 0
    #       Semi-open rows of length 5: 0
    #       White stones:
    #       Open rows of length 2: 0
    #       Semi-open rows of length 2: 0
    #       Open rows of length 3: 0
    #       Semi-open rows of length 3: 1
    #       Open rows of length 4: 0
    #       Semi-open rows of length 4: 0
    #       Open rows of length 5: 0
    #       Semi-open rows of length 5: 0

    y = 3; x = 5; d_x = -1; d_y = 1; length = 2

    put_seq_on_board(board, y, x, d_y, d_x, length, "b")
    print_board(board)
    analysis(board)

    # Expected output:
    #        *0|1|2|3|4|5|6|7*
    #        0 | | | | |w|b| *
    #        1 | | | | | | | *
    #        2 | | | | | | | *
    #        3 | | | | |b| | *
    #        4 | | | |b| | | *
    #        5 | |w| | | | | *
    #        6 | |w| | | | | *
    #        7 | |w| | | | | *
    #        *****************
    #
    #         Black stones:
    #         Open rows of length 2: 1
    #         Semi-open rows of length 2: 0
    #         Open rows of length 3: 0
    #         Semi-open rows of length 3: 0
    #         Open rows of length 4: 0
    #         Semi-open rows of length 4: 0
    #         Open rows of length 5: 0
    #         Semi-open rows of length 5: 0
    #         White stones:
    #         Open rows of length 2: 0
    #         Semi-open rows of length 2: 0
    #         Open rows of length 3: 0
    #         Semi-open rows of length 3: 1
    #         Open rows of length 4: 0
    #         Semi-open rows of length 4: 0
    #         Open rows of length 5: 0
    #         Semi-open rows of length 5: 0
    #

    y = 5; x = 3; d_x = -1; d_y = 1; length = 1
    put_seq_on_board(board, y, x, d_y, d_x, length, "b");
    print_board(board);
    analysis(board);

    #        Expected output:
    #           *0|1|2|3|4|5|6|7*
    #           0 | | | | |w|b| *
    #           1 | | | | | | | *
    #           2 | | | | | | | *
    #           3 | | | | |b| | *
    #           4 | | | |b| | | *
    #           5 | |w|b| | | | *
    #           6 | |w| | | | | *
    #           7 | |w| | | | | *
    #           *****************
    #
    #
    #        Black stones:
    #        Open rows of length 2: 0
    #        Semi-open rows of length 2: 0
    #        Open rows of length 3: 0
    #        Semi-open rows of length 3: 1
    #        Open rows of length 4: 0
    #        Semi-open rows of length 4: 0
    #        Open rows of length 5: 0
    #        Semi-open rows of length 5: 0
    #        White stones:
    #        Open rows of length 2: 0
    #        Semi-open rows of length 2: 0
    #        Open rows of length 3: 0
    #        Semi-open rows of length 3: 1
    #        Open rows of length 4: 0
    #        Semi-open rows of length 4: 0
    #        Open rows of length 5: 0
    #        Semi-open rows of length 5: 0




if __name__ == '__main__':
    easy_testset_for_main_functions()
    #some_tests()

TEST CASE for is_empty PASSED
*0|1|2|3|4|5|6|7*
0 | | | | |w| | *
1 | | | | |w| | *
2 | | | | |w| | *
3 | | | | |w| | *
4 | | | | | | | *
5 | | | | | | | *
6 | | | | | | | *
7 | | | | | | | *
*****************
TEST CASE for is_bounded PASSED
*0|1|2|3|4|5|6|7*
0 | | | | | | | *
1 | | | | |w| | *
2 | | | | |w| | *
3 | | | | |w| | *
4 | | | | | | | *
5 | | | | | | | *
6 | | | | | | | *
7 | | | | | | | *
*****************
TEST CASE for detect_row PASSED
*0|1|2|3|4|5|6|7*
0 | | | | | | | *
1 | | | | |w| | *
2 | | | | |w| | *
3 | | | | |w| | *
4 | | | | | | | *
5 | | | | | | | *
6 | | | | | | | *
7 | | | | | | | *
*****************
TEST CASE for detect_rows PASSED
*0|1|2|3|4|5|6|7*
0 | | | | |w|b| *
1 | | | | |w|b| *
2 | | | | |w|b| *
3 | | | | |w|b| *
4 | | | | | | | *
5 | | | | | | | *
6 | | | | | | | *
7 | | | | | | | *
*****************
(4, 0)
TEST CASE for search_max FAILED
